# Coursework 1: Question 2 - Vodacom Dataset

## Import Libraries

In [1]:
import pandas as pd
from tqdm.notebook import tqdm

## Source Data

Get the source data to use for clustering.

### Data Load

Load customers reviews that was previously downloaded from Hellopeter in the [DSM020](https://github.com/JohnnyFoulds/dsm020-2021-oct) git repository.

In [2]:
df_source = pd.read_parquet('https://github.com/JohnnyFoulds/dsm020-2021-oct/blob/master/coursework_01/data/output/telecommunications.gzip?raw=true')
df_source.shape

(35072, 17)

In [3]:
df_source.head(2)

,id,created_at,author,author_id,review_title,review_rating,review_content,business_slug,permalink,replied,messages,industry_slug,nps_rating,author_created_date,author_total_reviews_count,review_title_clean,review_content_clean
0,3750417,2021-12-21 15:27:08,Barry S,05c6e290-6186-11ec-b1c0-cd74559df45d,Worst Service,1,Worst service I’ve received in my life. Non of...,vodacom,worst-service-04099ff193227e2908f31413b00ffb30...,False,[],telecommunications,NaN,2021-12-20,1,Worst Service,Worst service I’ve received in my life. Non of...
1,3750406,2021-12-21 15:19:45,Kulani Marry-Aan,0a616360-6260-11ec-b18f-0f0735b462a0,Complaint about customer service,1,18/12/2021 i bought a router at Vodacom Bushbu...,vodacom,complaint-about-customer-service-28c9107344f3a...,False,[],telecommunications,NaN,2021-12-21,1,Complaint about customer service,18/12/2021 i bought a router at Vodacom Bushbu...


### Data Selection

Select only reviews from `vodacom` with the `review_title` and `review_content_clean` being the only data of interest.

In [4]:
df_reviews = df_source.query('business_slug == "vodacom"')[['id', 'review_title', 'review_content_clean']].reset_index(drop=True)
df_reviews.shape

(13699, 3)

In [5]:
df_reviews.head(3)

,id,review_title,review_content_clean
0,3750417,Worst Service,Worst service I’ve received in my life. Non of...
1,3750406,Complaint about customer service,18/12/2021 i bought a router at Vodacom Bushbu...
2,3750373,VODACOM not living up to promises and just wan...,I need help with Vodacom. I have been a custo...


### Data Preparation

Create an individual text file for each review for upload to HDFS.

In [10]:
def create_files(data:pd.DataFrame, output_path:str, id_column:str='id', title_column:str='review_title', content_column:str='review_content_clean') -> None:
    """
    Create a output .txt file for each review in the dataset.

    Parameters
    ----------
    data : pandas.DataFrame
        The DataFrame containint the reviews to process.

    output_path : str
        The path where the .txt files should be stored

    id_column : str
        The name of the ID column that will be used for the filename.

    title_column : str
        The name of the column in the DataFrame containing the review title.

    content_column : str
        The name of the column in the DataFrame containing the review text.
    """
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        # get the output filename
        filename = '%s/%s.txt' % (output_path, row[id_column])

        # get the text to write to file
        output_text = '%s\n\n%s' % (row[title_column], row[content_column])

        # create the output file
        with open(filename, 'w') as f:
            f.write(output_text)

# create the output files
create_files(df_reviews, 'data/input/vodacom')

  0%|          | 0/3 [00:00<?, ?it/s]